In [1]:
# # !pip install webrtcvad
# !pip install pygame
# !pip install pyaudio webrtcvad 
# !pip install google-cloud-texttospeech


In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq
from QuoteUtils import CtxMgr
from groq import Groq
import random
from langchain_groq import ChatGroq
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")

In [3]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool
import re

llm.groq_api_key = random.choice(api_keys)

def add_section(doc, text, font_name='Arial', font_size=12, bold=False, italic=False, alignment='left', color=None):
    # Check if the text should be a bullet point
    if text.startswith("- ") or text.startswith("+ "):
        paragraph = doc.add_paragraph(style='ListBullet')
        text = text[2:]  # Remove the bullet point indicator
    else:
        paragraph = doc.add_paragraph()
    
    run = paragraph.add_run(text)
    run.font.name = font_name
    run.font.size = Pt(font_size)
    run.bold = bold
    run.italic = italic
    if color:
        run.font.color.rgb = RGBColor(*color)
    
    if alignment == 'center':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT


def write_file(title, sections, append=False):
    most_recent_file_created_path = "most_recent_file_created.txt"
    if os.path.exists(most_recent_file_created_path):
        with open(most_recent_file_created_path, 'r') as mostrec:
            most_recent_title = mostrec.read().strip()
        if most_recent_title:
            
            if most_recent_title != title:
                title = most_recent_title
        else:
            pass
    else:
        print("Most recent file created does not exist.")
    
    
        

    documents_path = os.path.join(os.getcwd(), 'documents')
    os.makedirs(documents_path, exist_ok=True)
    docx_path = os.path.join(documents_path, f'{title}.docx')
    pdf_path = os.path.join(documents_path, f'{title}.pdf')
    
    with open(most_recent_file_created_path,'w') as mostrec:
        mostrec.write(title)
        
    if append and os.path.exists(docx_path):
        doc = Document(docx_path)
    else:
        doc = Document()

    for section in sections:
        add_section(
            doc,
            section["content"],
            font_name=section.get("font_name", "Arial"),
            font_size=section.get("font_size", 12),
            bold=section.get("bold", False),
            italic=section.get("italic", False),
            alignment=section.get("alignment", "left"),
            color=section.get("color", None)
        )
    
    doc.save(docx_path)
    convert(docx_path, pdf_path)

    return f"Document written and converted to pdf. No further action needed. STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE! YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!"


class WriteFileTool(BaseTool):
    name = "write_file"
    description = "Writes content into a .docx file with specified formatting and converts it to a .pdf file.\
        If you encounter errors, use this tool ONLY and input everything properly and try again.\
            IMPORTANT! - STOP AFTER ONE SUCCESSFUL EXECUTION! STOP AFTER ONE SUCCESSFUL EXECUTION! STOP AFTER ONE SUCCESSFUL EXECUTION! "
    
    def _run(self, title, sections, append=False):
        self.process_sections_in_batches(title, sections, append)
        return ("STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE! YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!")

    def process_sections_in_batches(self, title, sections, append, batch_size=1):
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i+batch_size]
            result = write_file(title, batch, append=(i > 0))
        



def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        if content_match:
            content = content_match.group(1)
            # Replace double quotes indicating inches with the word "inch"
            content = re.sub(r'(?<=\d)"', ' inch', content)
            section["content"] = content
        else:
            section["content"] = ""
        
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append


# cc_out = """Emotional Intelligence (EI) refers to the ability to recognize and understand emotions in oneself and others. It involves being aware of one's emotions and how they impact behavior, as well as being able to manage and regulate those emotions to achieve goals and maintain healthy relationships. People with high emotional intelligence are better equipped to navigate complex social situations, build stronger relationships, and make more informed decisions.

# Developing emotional intelligence can have numerous benefits, both personally and professionally. It can lead to improved communication, increased empathy, and enhanced teamwork and leadership skills. Additionally, high emotional intelligence has been linked to better stress management, increased resilience, and improved overall well-being. By cultivating emotional awareness, self-regulation, and social skills, individuals can unlock their full potential and achieve greater success in all areas of life."""


def make_docmgr_write_to_file(cc_out):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a detailed and professional assistant. Your task is to generate comprehensive and detailed formatted text. 
                Ensure everything is included without summarizing or truncating any content.
                Your response must be well-formed and include all details. 

                You must ALWAYS ALWAYS ALWAYS!!!!! output in a structured format as follows AND NO QUOTATION MARKS OR BACKSLASHES!:
                
                            
                {
                    "title": "The Great Renovation",
                    "sections": [
                        {
                            "content": "<specific text content>",
                            "font_name": "<font family>",
                            "font_size": <integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>"
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        },
                        {
                            "content": "<another specific text content>",
                            "font_name": "<another font family>",
                            "font_size": <another integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>"
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        }
                        // ... more sections as needed
                    ],
                    "append": <true or false> (this indicates appending to document or not)
                }
                PLEASE! Ensure the output is well-formed and valid."""
            },
            {
                "role": "user",
                "content": "Give whatever file title you want;NO COLONS!. Please Include a new title IN THE DOCUMENT, headings if you deem fit, and please format in a nice readable and coherent way.  \
                    Specify the formatting for this text. Please make it a \
                        PROFESSIONALLY formatted text WITH black & blue colors and times new roman font PLEASE. NO NO NO bullet points of ANY KIND unless specified: " + cc_out
            }
        ],
        model="llama3-70b-8192",
    )

    cc_out2 = chat_completion.choices[0].message.content

    print('CCOUT2\n',cc_out2)

    # title, sections, append = parse_formatted_response(cc_out2)

    # WriteFileTool()._run(title, sections, append)

    return cc_out2

#make_docmgr_write_to_file()


c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.tools import BaseTool
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import googleapiclient
import google.auth.exceptions

class GoogleDriveUploadTool(BaseTool):
    name = "GoogleDriveUploadTool"
    description = ("Uploads a PDF to Google Drive and sets permissions for a specific user. "
                   "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
                   "to Google Drive. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")

    def _run(self, file_path: str, user_email: str, rename: str) -> str:
        credentials_path = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json"

        # Authenticate and create the service
        try:
            credentials = service_account.Credentials.from_service_account_file(credentials_path)
            service = build('drive', 'v3', credentials=credentials)
            print("Successfully authenticated and created the service.")
        except google.auth.exceptions.GoogleAuthError as e:
            return f"Authentication error: {e}"

        # File details
        if rename is not None:
            file_metadata = {'name': rename}
        else:
            file_metadata = {'name': os.path.basename(file_path)}

        if not os.path.exists(file_path):
            return f"File not found: {file_path}"
        else:
            media = MediaFileUpload(file_path, mimetype='application/pdf')
            try:
                # Upload file
                file = service.files().create(body=file_metadata, media_body=media, fields='id, name, parents').execute()
                file_id = file.get('id')
                with open('file_id_history.txt','w') as id_hist_file:
                    id_hist_file.write(file_id)
                print(f"File ID: {file_id}")

                # Verify the upload by fetching the file details
                file_info = service.files().get(fileId=file_id, fields='id, name, parents').execute()
                print("File details:")
                print(file_info)

                # Sharing settings
                permission = {
                    'type': 'user',
                    'role': 'writer',  # Set to 'reader' if read-only access is needed
                    'emailAddress': user_email,
                    
                }
                

                # Grant permission to the specific user
                try:
                    service.permissions().create(
                        fileId=file_id,
                        body=permission,
                        fields='id',
                        sendNotificationEmail=False
                    ).execute()
                    
                    print(f"Granted {permission['role']} access to {user_email}")
                    
                    

                except googleapiclient.errors.HttpError as e:
                    print(f"Error setting permission: {e}")
                    return f"Error setting permission: {e}"

                return f"File uploaded and permissions set successfully. File ID: {file_id}. PLEASE STOP WORKING! THE TOOL HAS BEEN EXECUTED PROPERLY!" 

            except Exception as e:
                return f"Error during file upload or conversion: {e}"

    

    def _arun(self, file_path: str, user_email: str, rename: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

# Example usage
# tool = GoogleDriveUploadTool()
# response = tool._run(file_path='./documents/The Great Renovation.pdf', user_email='specific-user@example.com', rename=None)
# print(response)

import os
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build


class GoogleSheetsUpdateTool(BaseTool):
    name = "GoogleSheetsUpdateTool"
    description = ("Appends three columns - name, phone number, and link to a PDF - to a preexisting logging google sheet. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    credentials_path = 'C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json'
    spreadsheet_id = '1TSFyiTwctC1tABr2RQouBzLRMCG4RZ7lXdTVi-I58Mo'
    range_name = 'Sheet1'
    
    #creds = authenticate_google_sheets()
    
    def authenticate_google_sheets(self):
        creds = service_account.Credentials.from_service_account_file(
            self.credentials_path, scopes=self.SCOPES)
        return creds
    
    def read_file_content(self, file_path):
        with open(file_path, 'r') as file:
            content = file.read().strip()
        return content
    
    # def extract_file_id(self,output):
    #     match = re.search(r"File ID: ([^'}]+)", output)
    #     if match:
    #         file_id = match.group(1).strip()
    #         return file_id
    #     else:
    #         raise ValueError("File ID not found in the inputted string")
    
    def append_row_to_google_sheets(self, values):
        """
        Appends a row to the specified range in the Google Sheets spreadsheet.
        
        :param values: The data to append, as a list of lists (e.g., [['A1', 'B1', 'C1']]).
        """
        service = build('sheets', 'v4', credentials=self.authenticate_google_sheets())
        sheet = service.spreadsheets()
        body = {'values': values}
        result = sheet.values().append(
            spreadsheetId=self.spreadsheet_id, range=self.range_name,
            valueInputOption="RAW", body=body,
            insertDataOption="INSERT_ROWS"
        ).execute()
        print(f"{result.get('updates').get('updatedCells')} cells appended.")

    def _run(self, name: str, phone_number: str, linkstr:str):
        """
        Run the tool to append the row with the given name, phone number, and link to the PDF.
        
        :param name: The full legal name to append.
        :param phone_number: The phone number to append.
        """
        fileID = self.read_file_content('file_id_history.txt')
        
        print(f"Extracted file ID: {fileID}")
        link = f"https://drive.google.com/file/d/{fileID}/view"
        values = [[]]
        if linkstr:
            values = [[name, phone_number, linkstr]]
        else:
            values = [[name, phone_number, link]]
        self.append_row_to_google_sheets(values)
        return "Row appended successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!"

    def _arun(self, name: str, phone_number: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")
    
import os
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from langchain.tools import BaseTool
from pydantic import BaseModel, Field, Extra

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

class GmailSendPdfTool(BaseTool):
    name = "GmailSendPdfTool"
    description = "Sends an email with an optional PDF attachment using Gmail API."

    credentials_path: str = Field(..., description="Path to the credentials JSON file")

    class Config:
        extra = Extra.allow

    def __init__(self, credentials_path: str):
        super().__init__()
        self.credentials_path = credentials_path
        self.creds = None
        self.service = None
        self.authenticate()

    def authenticate(self):
        """Authenticate the user with Gmail API."""
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
            
        self.service = build('gmail', 'v1', credentials=self.creds)

    def send_email(self, sender_email, recipient_email, subject, body, pdf_path=None):
        """Send an email with an optional PDF attachment."""
        message = MIMEMultipart()
        message['to'] = recipient_email
        message['from'] = sender_email
        message['subject'] = subject
        body_part = MIMEText(body, 'plain')
        message.attach(body_part)
        
        if pdf_path:
            part = MIMEBase('application', 'octet-stream')
            with open(pdf_path, 'rb') as file:
                part.set_payload(file.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(pdf_path)}"')
            message.attach(part)
       
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        raw_message = {'raw': raw_message}

        try:
            message = (self.service.users().messages().send(userId="me", body=raw_message).execute())
            print(f'Message Id: {message["id"]}')
            return message
        except HttpError as error:
            print(f'An error occurred: {error}')
            return None

    def _run(self, sender_email: str, recipient_email: str, subject: str, body: str, pdf_path: str = None):
        """Run the tool to send the email with the optional PDF attachment."""
        result = self.send_email(sender_email, recipient_email, subject, body, pdf_path)
        return "Email sent successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!" if result else "Failed to send email."

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")



C:\Users\pc-user1\AppData\Local\Temp\ipykernel_10804\4267615534.py:191: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  extra = Extra.allow


In [6]:
from flask import Flask, request, jsonify, send_file, render_template
import whisper
import pyaudio
import wave
import webrtcvad
import collections
import base64
from google.cloud import texttospeech
import json
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from aiogoogle import Aiogoogle
from flask_cors import CORS

import json
from langchain.agents import Tool
my_tools = []



my_tools.append(
    
    WriteFileTool()
    
)

my_tools.append(
    
    GoogleDriveUploadTool()
    
)

my_tools.append(
    
    GoogleSheetsUpdateTool()
    
)
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
my_tools.append(
    GmailSendPdfTool(credentials_path)
)




llm.groq_api_key = random.choice(api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)



from flask import Flask, request, jsonify, send_file, render_template
import whisper
import pyaudio
import wave
import webrtcvad
import collections
import base64
from google.cloud import texttospeech
import json
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
from flask_cors import CORS
import uuid
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(threadName)s] %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Global variables
chat_history = []
model = whisper.load_model("base")
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 1000000
vad = webrtcvad.Vad(3)
audio = pyaudio.PyAudio()
credentials = {"name": "", "email": "", "recemail": "", "phone": ""}

# Initialize text-to-speech client
tts_client = texttospeech.TextToSpeechClient.from_service_account_file("C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json")

executor = ThreadPoolExecutor(max_workers=5)

def record_audio():
    logger.debug('Starting audio recording...')
    try:
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                            rate=RATE, input=True,
                            frames_per_buffer=CHUNK)
        frames = []
        ring_buffer = collections.deque(maxlen=100)
        triggered = False
        voiced_frames = []
        silence_threshold = 10
        silence_chunks = 0

        for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)

            num_subframes = int(len(data) / 320)
            for i in range(num_subframes):
                subframe = data[i*320:(i+1)*320]
                is_speech = vad.is_speech(subframe, RATE)
                ring_buffer.append((subframe, is_speech))

            num_voiced = len([f for f, speech in ring_buffer if speech])

            if not triggered:
                if num_voiced > 0.6 * ring_buffer.maxlen:
                    triggered = True
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(data)
                if num_voiced < 0.2 * ring_buffer.maxlen:
                    silence_chunks += 1
                    if silence_chunks > silence_threshold:
                        triggered = False
                        break
                else:
                    silence_chunks = 0

        stream.stop_stream()
        stream.close()

        with wave.open("C:\\DEV\\webdevfolder\\output.wav", 'wb') as wf:
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(audio.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(voiced_frames))
        logger.debug('Audio recording completed and file saved.')
    except Exception as e:
        logger.error(f"An error occurred while recording audio: {e}")

def transcribe_audio():
    result = model.transcribe("C:\\DEV\\webdevfolder\\output.wav")
    transcription = result['text']
    
    logger.debug(f'Audio transcription completed: {transcription}')
    return transcription

async def ai_response(transcription: str):
    global chat_history
    logger.debug(f'Generating AI response for transcription: {transcription}')
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a nice, great assistant. User will tell you things. You just respond. 
                If user's query relates to the following: you delegate the tasks to certain tools - namely, writing to a file, sending an email, uploading to google drive, or updating a google sheet. 
                If what the user says is one of these three you must explicitly say AT THE END OF YOUR RESPONSE in this very format depending on which tool - "I will use the [write file tool, gmail tool, sheets tool, OR drive upload tool]". 
                so that user can confirm if you got it correctly. 
                When writing to a file, just start the content with '@' symbol when generating file content.
                IMPORTANT: ONCE YOU HAVE EXECUTED THE TOOL YOU MUST SAY THAT IT EXECUTED SUCCESSFULLY. 
                IMPORTANT: IF YOU REMEMBER YOU HAVE USED A TOOL ALREADY (REFER TO CHAT HISTORY), AND YOU ARE IN THE PROCESS OF USING IT AGAIN, \
                YOU MUST ASK USER IF THEY ARE SURE TO USE THE TOOL AGAIN OR NOT.

                If you have to use a google related tool, you MUST prompt the user or remind user to enter credentials in the doc manager extension. 
                If user tells you to do something that is not one of these, you kindly say that you don't have access to that functionality.
                """
            },
            {
                "role": "user",
                "content": transcription + "Here is the chat history for context (NEVER TALK ABOUT CHAT HISTORY IT IS ONLY FOR YOU NEVER TALK ABOUT IT IN YOUR RESPONSES!!!!): [" + str(chat_history) + "]"
            }
        ],
        model="llama3-70b-8192",
    )
    response = chat_completion.choices[0].message.content
    logger.debug(f'AI response generated: {response}')
    chat_history.append("USER: " + transcription + "\nTHE AI MODEL: " + response + "\n")
    
    
    asyncio.create_task(handle_response_with_agents(response))
   
    await synthesize_speech(response)

    return response

async def handle_response_with_agents(response):
    logger.debug(f'Handling response with agents: {response}')
    await asyncio.get_event_loop().run_in_executor(executor, handle_agents, response)

def handle_agents(response):
    logger.debug(f'Processing response with agents: {response}')
    name = credentials['name']
    email = credentials['email']
    recemail = credentials["recemail"]
    phone = credentials['phone']
    print(email, recemail, name, phone)

    title_of_file = ''
    with open('most_recent_file_created.txt') as mostrec:
        title_of_file = mostrec.read()
    thepath = f"./documents/{title_of_file}.pdf"

    if 'I will use' in response:
        if 'write file' in response:
            idx = response.find('@')
            modified_response = response[idx + 1:].strip()
            print('MODIFIED RESPONSE = ', modified_response)
            cc_out2 = make_docmgr_write_to_file(modified_response)
            title, sections, append = parse_formatted_response(cc_out2)
            llm.groq_api_key = random.choice(api_keys)
            agent.invoke({"input": f"use ONLY write_file tool!!! INPUT EVERYTHING CORRECTLY!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! append-->true or false lowercase only!!! --> TITLE: {title}, SECTIONS: {json.dumps(sections)}, APPEND: {append}"})
        elif 'sheet' in response:
            agent.invoke({"input": f"please ONLY ONLY use google sheets update tool, input ONLY ONLY these into the tool PROPERLY! --> {credentials['name']}, {credentials['phone']}"})
        elif 'upload' in response:
            agent.invoke({"input": f"please use google drive upload tool, INPUTS --> FILE_PATH: {thepath} USER_EMAIL: {credentials['email']}"})
        elif 'mail' in response:
            if 'pdf' in response or 'PDF' in response or 'file' in response:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}"})
            else:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing"})

async def synthesize_speech(text):
    logger.debug(f'Starting speech synthesis for text: {text}')
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Casual-K"
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = await asyncio.get_event_loop().run_in_executor(
        executor, lambda: tts_client.synthesize_speech(
            input=synthesis_input, voice=voice, audio_config=audio_config
        )
    )
    async with aiofiles.open("C:\\DEV\\webdevfolder\\synthesis.mp3", 'wb') as out:
        await out.write(response.audio_content)
    print('Audio content written to file "synthesis.mp3"')
    logger.debug('Speech synthesis completed and file saved.')


@app.route('/set_credentials', methods=['POST'])
def set_credentials():
    global credentials
    data = request.get_json()
    if not data:
        return jsonify({"status": "failed", "message": "No data received"}), 400
    credentials['name'] = data.get('name')
    credentials['email'] = data.get('email')
    credentials['recemail'] = data.get('recemail')
    credentials['phone'] = data.get('phone')
    print("THE CREDENTIALS ****** -------------> ", credentials)
    return jsonify({"status": "success"})

@app.route('/')
def index():
    return render_template('index2.html')

@app.route('/talk', methods=['POST'])
async def talk():
    loop = asyncio.get_event_loop()
    
    logger.debug('Starting audio recording...')
    await loop.run_in_executor(executor, record_audio)
    logger.debug('Audio recording completed.')
    
    logger.debug('Starting audio transcription...')
    transcription = await loop.run_in_executor(executor, transcribe_audio)
    logger.debug(f'Audio transcription completed: {transcription}')
    
    logger.debug('Generating AI response...')
    ai_resp = await ai_response(transcription)
    logger.debug(f'AI response generated: {ai_resp}')
    
    return jsonify({'response': ai_resp})



@app.route('/get_audio')
def get_audio():
    return send_file("C:\\DEV\\webdevfolder\\synthesis.mp3", mimetype="audio/mp3")

if __name__ == '__main__':
    app.run()



2024-06-01 21:34:12,066 [MainThread] DEBUG: Generated new state NDVJy4Q0C70URfTbGSwsfcJzCo30no.


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64282%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=NDVJy4Q0C70URfTbGSwsfcJzCo30no&access_type=offline


2024-06-01 21:34:20,203 [MainThread] INFO: "GET /?state=NDVJy4Q0C70URfTbGSwsfcJzCo30no&code=4/0AdLIrYelU1vLf_B9KzYRYmOKf7BPtFYCXjFtg9rRuuVZAQFBGChscgYHHbttwhQ6bIBp7Q&scope=https://www.googleapis.com/auth/gmail.send HTTP/1.1" 200 65
2024-06-01 21:34:20,203 [MainThread] DEBUG: Encoding `client_id` "291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com" with `client_secret` as Basic auth credentials.
2024-06-01 21:34:20,203 [MainThread] DEBUG: Requesting url https://oauth2.googleapis.com/token using method POST.
2024-06-01 21:34:20,203 [MainThread] DEBUG: Supplying headers {'Accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded'} and data {'grant_type': 'authorization_code', 'code': '4/0AdLIrYelU1vLf_B9KzYRYmOKf7BPtFYCXjFtg9rRuuVZAQFBGChscgYHHbttwhQ6bIBp7Q', 'redirect_uri': 'http://localhost:64282/'}
2024-06-01 21:34:20,203 [MainThread] DEBUG: Passing through key word arguments {'timeout': None, 'auth': <requests.auth.HTTPBasicAuth object at 0x

 * Serving Flask app '__main__'
 * Debug mode: off


2024-06-01 21:34:21,850 [MainThread] INFO: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2024-06-01 21:34:21,850 [MainThread] INFO: Press CTRL+C to quit
2024-06-01 21:34:24,453 [Thread-8 (process_request_thread)] INFO: 127.0.0.1 - - [01/Jun/2024 21:34:24] "GET / HTTP/1.1" 200 -
2024-06-01 21:34:24,484 [Thread-9 (process_request_thread)] INFO: 127.0.0.1 - - [01/Jun/2024 21:34:24] "GET /static/favicon.ico HTTP/1.1" 304 -
2024-06-01 21:34:42,086 [Thread-10 (process_request_thread)] DEBUG: Using selector: SelectSelector
2024-06-01 21:34:42,086 [ThreadPoolExecutor-4_0] DEBUG: Starting audio recording...
2024-06-01 21:34:42,094 [ThreadPoolExecutor-3_0] DEBUG: Starting audio recording...
2024-06-01 21:34:46,429 [ThreadPoolExecutor-3_0] DEBUG: Audio recording completed and file saved.
2024-06-01 21:34:46,429 [ThreadPoolExecutor-4_0] DEBUG: Audio recording completed.
2024-06-01 21:34:46,

...

2024-06-01 21:36:26,459 [ThreadPoolExecutor-3_0] DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 02 Jun 2024 01:36:16 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Cache-Control', b'private, max-age=0, no-store, no-cache, must-revalidate'), (b'vary', b'Origin'), (b'x-ratelimit-limit-requests', b'14400'), (b'x-ratelimit-limit-tokens', b'6000'), (b'x-ratelimit-remaining-requests', b'14398'), (b'x-ratelimit-remaining-tokens', b'5273'), (b'x-ratelimit-reset-requests', b'11.224999999s'), (b'x-ratelimit-reset-tokens', b'7.27s'), (b'x-request-id', b'req_01hzb9e3yhfkpact09pc7rv6qs'), (b'via', b'1.1 google'), (b'alt-svc', b'h3=":443"; ma=86400'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'88d3b61f5cc207fb-IAD'), (b'Content-Encoding', b'gzip')])
2024-06-01 21:36:26,464 [ThreadPoolExecutor-3_0] INFO: HTTP Request: POST https://api.groq.com/open

CCOUT2
 Here is the formatted text:

{
"title": "AI Conversational Revolution",
"sections": [
{
"content": "AI Conversational Revolution",
"font_name": "Times New Roman",
"font_size": 24,
"bold": true,
"italic": false,
"alignment": "center",
"color": [0, 0, 0]
},
{
"content": "Chat GPT: The Future of Conversational AI",
"font_name": "Times New Roman",
"font_size": 18,
"bold": true,
"italic": false,
"alignment": "center",
"color": [0, 0, 255]
},
{
"content": "Chat GPT is a revolutionary artificial intelligence model that has been designed to engage in natural-sounding conversations with humans.",
"font_name": "Times New Roman",
"font_size": 12,
"bold": false,
"italic": false,
"alignment": "justify",
"color": [0, 0, 0]
},
{
"content": "It uses a massive language model that is trained on a vast amount of text data, allowing it to understand and respond to a wide range of questions and topics.",
"font_name": "Times New Roman",
"font_size": 12,
"bold": false,
"italic": false,
"alignment": "

2024-06-01 21:36:28,351 [ThreadPoolExecutor-3_0] DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Sun, 02 Jun 2024 01:36:18 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'Cache-Control', b'private, max-age=0, no-store, no-cache, must-revalidate'), (b'vary', b'Origin'), (b'x-ratelimit-limit-requests', b'14400'), (b'x-ratelimit-limit-tokens', b'6000'), (b'x-ratelimit-remaining-requests', b'14397'), (b'x-ratelimit-remaining-tokens', b'4939'), (b'x-ratelimit-reset-requests', b'16.385999999s'), (b'x-ratelimit-reset-tokens', b'10.61s'), (b'x-request-id', b'req_01hzb9e5k7f5avz84cyt9tycpn'), (b'via', b'1.1 google'), (b'alt-svc', b'h3=":443"; ma=86400'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'88d3b629f9cb7ffd-IAD'), (b'Content-Encoding', b'gzip')])
2024-06-01 21:36:28,351 [ThreadPoolExecutor-3_0] INFO: HTTP Request: POST https://api.groq.com/ope

Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "AI Conversational Revolution",
    "sections": [{"content": "AI Conversational Revolution", "font_name": "Times New Roman", "font_size": 24, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Chat GPT: The Future of Conversational AI", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 255]}, {"content": "Chat GPT is a revolutionary artificial intelligence model that has been designed to engage in natural-sounding conversations with humans.", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "justify", "color": [0, 0, 0]}, {"content": "It uses a massive language model that is trained on a vast amount of text data, allowing it to understand and respond to a wide range of questions and topics.", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": 

100%|██████████| 1/1 [00:00<00:00,  1.33it/s]